In [ ]:
import json
import pandas as pd
from typing import List, Dict
import openai
from tqdm import tqdm

class RAGEvaluator:
    def __init__(self, openai_client, vector_store, rag_function):
        """
        Initialize evaluator
        
        Args:
            openai_client: OpenAI client instance
            vector_store: Your ArizonaPlantVectorStore instance
            rag_function: Your rag() function
        """
        self.client = openai_client
        self.vector_store = vector_store
        self.rag = rag_function
    
    # ============================================================
    # PART 1: Generate Ground Truth Questions
    # ============================================================
    
    def generate_questions_for_document(self, document: Dict, num_questions: int = 3) -> List[Dict]:
        """
        Generate questions for a single document
        
        Args:
            document: Document dict with 'content', 'title', etc.
            num_questions: Number of questions to generate
            
        Returns:
            List of question-answer pairs
        """
        content = document.get('content', '')[:2000]  # Limit context
        title = document.get('title', 'Unknown')
        
        prompt = f"""Based on the following document about Arizona desert plants, generate {num_questions} realistic questions that a gardener or plant enthusiast might ask.

Document Title: {title}
Document Content:
{content}

Generate {num_questions} questions that:
1. Are specific and answerable from this document
2. Represent realistic user queries (mix of detailed and simple)
3. Include scientific and common plant names when relevant
4. Focus on practical gardening concerns (care, identification, selection)

Return ONLY a JSON array of questions, like this:
["Question 1?", "Question 2?", "Question 3?"]

Questions:"""
        
        response = self.client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8  # More creative for diverse questions
        )
        
        questions_text = response.choices[0].message.content.strip()
        
        # Parse JSON
        try:
            # Remove markdown code blocks if present
            if '```' in questions_text:
                questions_text = questions_text.split('```')[1]
                if questions_text.startswith('json'):
                    questions_text = questions_text[4:]
            
            questions = json.loads(questions_text)
        except:
            # Fallback parsing
            questions = [q.strip() for q in questions_text.split('\n') if q.strip() and '?' in q]
        
        # Generate answers for each question
        qa_pairs = []
        for question in questions[:num_questions]:
            answer_prompt = f"""Based ONLY on the following document, answer this question concisely:

Document: {content}

Question: {question}

Answer (2-3 sentences):"""
            
            answer_response = self.client.chat.completions.create(
                model='gpt-4o-mini',
                messages=[{"role": "user", "content": answer_prompt}],
                temperature=0.3  # More factual
            )
            
            answer = answer_response.choices[0].message.content.strip()
            
            qa_pairs.append({
                'question': question,
                'ground_truth_answer': answer,
                'source_doc_id': document.get('id'),
                'source_doc_title': title,
                'document_type': document.get('type')
            })
        
        return qa_pairs
    
    def generate_ground_truth_dataset(self, documents: List[Dict], 
                                     questions_per_doc: int = 3,
                                     max_documents: int = 50) -> List[Dict]:
        """
        Generate ground truth Q&A dataset from your documents
        
        Args:
            documents: List of document dicts
            questions_per_doc: Questions to generate per document
            max_documents: Limit number of documents to process
            
        Returns:
            List of Q&A pairs with metadata
        """
        print(f"Generating ground truth dataset from {min(len(documents), max_documents)} documents...")
        
        all_qa_pairs = []
        
        # Sample diverse documents
        docs_to_process = documents[:max_documents]
        
        for doc in tqdm(docs_to_process, desc="Generating questions"):
            try:
                qa_pairs = self.generate_questions_for_document(doc, questions_per_doc)
                all_qa_pairs.extend(qa_pairs)
            except Exception as e:
                print(f"Error processing document {doc.get('id')}: {e}")
                continue
        
        print(f"✓ Generated {len(all_qa_pairs)} question-answer pairs")
        return all_qa_pairs
    
    # ============================================================
    # PART 2: Retrieval Evaluation
    # ============================================================
    
    def evaluate_retrieval(self, qa_pair: Dict, top_k: int = 5) -> Dict:
        """
        Evaluate if retrieval finds the correct source document
        
        Args:
            qa_pair: Dict with 'question' and 'source_doc_id'
            top_k: Number of results to check
            
        Returns:
            Dict with retrieval metrics
        """
        question = qa_pair['question']
        expected_doc_id = qa_pair['source_doc_id']
        
        # Get search results
        results = self.vector_store.search(question, limit=top_k)
        
        # Check if expected doc is in results
        retrieved_ids = [r.get('id') for r in results]
        
        hit = expected_doc_id in retrieved_ids
        
        # Find rank (position) if present
        rank = None
        if hit:
            rank = retrieved_ids.index(expected_doc_id) + 1  # 1-indexed
        
        # Reciprocal rank for MRR calculation
        reciprocal_rank = 1.0 / rank if rank else 0.0
        
        return {
            'hit': hit,
            'rank': rank,
            'reciprocal_rank': reciprocal_rank,
            'retrieved_ids': retrieved_ids[:3],  # Top 3 for inspection
            'top_score': results[0]['score'] if results else 0.0
        }
    
    # ============================================================
    # PART 3: Answer Quality Evaluation (LLM-as-Judge)
    # ============================================================
    
    def evaluate_answer_quality(self, question: str, generated_answer: str, 
                                ground_truth_answer: str, retrieved_docs: List[Dict]) -> Dict:
        """
        Use LLM to judge answer quality on multiple criteria
        
        Args:
            question: User's question
            generated_answer: Your RAG system's answer
            ground_truth_answer: Expected answer
            retrieved_docs: Documents retrieved by your system
            
        Returns:
            Dict with quality scores
        """
        # Build context from retrieved docs
        context = "\n\n".join([
            f"Doc {i+1}: {doc.get('content', '')[:500]}"
            for i, doc in enumerate(retrieved_docs[:3])
        ])
        
        evaluation_prompt = f"""You are evaluating a question-answering system about Arizona desert plants.

Question: {question}

Retrieved Context:
{context}

Generated Answer: {generated_answer}

Ground Truth Answer: {ground_truth_answer}

Evaluate the Generated Answer on these criteria (rate 1-5):

1. RELEVANCE: Does it directly answer the question?
2. FAITHFULNESS: Is it grounded in the retrieved context (no hallucinations)?
3. COMPLETENESS: Does it cover key points from ground truth?
4. CLARITY: Is it well-written and easy to understand?

Respond ONLY with a JSON object:
{{
    "relevance": <score 1-5>,
    "faithfulness": <score 1-5>,
    "completeness": <score 1-5>,
    "clarity": <score 1-5>,
    "reasoning": "<brief explanation>"
}}"""
        
        response = self.client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{"role": "user", "content": evaluation_prompt}],
            temperature=0.2
        )
        
        eval_text = response.choices[0].message.content.strip()
        
        # Parse JSON
        try:
            if '```' in eval_text:
                eval_text = eval_text.split('```')[1]
                if eval_text.startswith('json'):
                    eval_text = eval_text[4:]
            
            scores = json.loads(eval_text)
        except:
            # Fallback if parsing fails
            scores = {
                'relevance': 3,
                'faithfulness': 3,
                'completeness': 3,
                'clarity': 3,
                'reasoning': 'Parse error'
            }
        
        # Calculate average score
        scores['average_score'] = (
            scores['relevance'] + 
            scores['faithfulness'] + 
            scores['completeness'] + 
            scores['clarity']
        ) / 4.0
        
        return scores
    
    # ============================================================
    # PART 4: Complete Evaluation Pipeline
    # ============================================================
    
    def evaluate_rag_system(self, ground_truth_dataset: List[Dict], 
                           save_results: bool = True) -> pd.DataFrame:
        """
        Run complete evaluation on your RAG system
        
        Args:
            ground_truth_dataset: List of Q&A pairs
            save_results: Whether to save detailed results to file
            
        Returns:
            DataFrame with evaluation results
        """
        print(f"\nEvaluating RAG system on {len(ground_truth_dataset)} questions...")
        print("="*60)
        
        results = []
        
        for i, qa_pair in enumerate(tqdm(ground_truth_dataset, desc="Evaluating"), 1):
            question = qa_pair['question']
            
            try:
                # 1. Evaluate retrieval
                retrieval_metrics = self.evaluate_retrieval(qa_pair, top_k=5)
                
                # 2. Generate answer with RAG
                generated_answer = self.rag(question)
                
                # 3. Get retrieved docs for context
                retrieved_docs = self.vector_store.search(question, limit=5)
                
                # 4. Evaluate answer quality
                quality_metrics = self.evaluate_answer_quality(
                    question=question,
                    generated_answer=generated_answer,
                    ground_truth_answer=qa_pair['ground_truth_answer'],
                    retrieved_docs=retrieved_docs
                )
                
                # Combine all metrics
                result = {
                    'question': question,
                    'ground_truth_answer': qa_pair['ground_truth_answer'],
                    'generated_answer': generated_answer,
                    'source_doc_id': qa_pair['source_doc_id'],
                    'source_doc_title': qa_pair.get('source_doc_title'),
                    
                    # Retrieval metrics
                    'retrieval_hit': retrieval_metrics['hit'],
                    'retrieval_rank': retrieval_metrics['rank'],
                    'retrieval_reciprocal_rank': retrieval_metrics['reciprocal_rank'],
                    
                    # Answer quality metrics
                    'relevance': quality_metrics['relevance'],
                    'faithfulness': quality_metrics['faithfulness'],
                    'completeness': quality_metrics['completeness'],
                    'clarity': quality_metrics['clarity'],
                    'average_score': quality_metrics['average_score'],
                    'reasoning': quality_metrics.get('reasoning', '')
                }
                
                results.append(result)
                
            except Exception as e:
                print(f"\nError evaluating question {i}: {e}")
                continue
        
        # Convert to DataFrame
        df_results = pd.DataFrame(results)
        
        # Calculate aggregate metrics
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        
        print("\n📊 Retrieval Metrics:")
        print(f"  Hit Rate@5: {df_results['retrieval_hit'].mean():.2%}")
        print(f"  Mean Reciprocal Rank: {df_results['retrieval_reciprocal_rank'].mean():.3f}")
        
        print("\n📝 Answer Quality (Average scores out of 5):")
        print(f"  Relevance:     {df_results['relevance'].mean():.2f}")
        print(f"  Faithfulness:  {df_results['faithfulness'].mean():.2f}")
        print(f"  Completeness:  {df_results['completeness'].mean():.2f}")
        print(f"  Clarity:       {df_results['clarity'].mean():.2f}")
        print(f"  Overall:       {df_results['average_score'].mean():.2f}")
        
        if save_results:
            df_results.to_csv('evaluation/rag_evaluation_results.csv', index=False)
            print(f"\n✓ Detailed results saved to: evaluation/rag_evaluation_results.csv")
        
        return df_results

In [45]:
# Run to reload modules automatically

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import os
print(os.getcwd())


/Users/pachon/training/llm-zoomcamp-class/arizona-desert-plants-chat


In [18]:
from openai import OpenAI

openai_client = OpenAI()

In [44]:
import sys
sys.path.append('assistant-app')
sys.path.append('../assistant-app')

from ArizonaPlantVectorStore import ArizonaPlantVectorStore
from ArizonaPlantRAG import ArizonaPlantRAG

In [43]:
from pathlib import Path
from typing import List, Dict

def load_dataset(dataset_path: str) -> List[Dict]:
    """Load the unified dataset"""
    print(f"\n3. Loading dataset from: {dataset_path}")
    
    path = Path(dataset_path)
    
    if not path.exists():
        raise FileNotFoundError(f"Dataset not found: {dataset_path}")
    
    # Handle different file formats
    if path.suffix == '.json':
        with open(path, 'r', encoding='utf-8') as f:
            documents = json.load(f)
    elif path.suffix == '.jsonl':
        documents = []
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                documents.append(json.loads(line))
    else:
        raise ValueError(f"Unsupported file format: {path.suffix}")
    
    print(f"   ✓ Loaded {len(documents)} documents")
    return documents

# Ground truth generation
We need to leverage the documents we created/generated during the data preparation. Therefore, we are going to be using the ```arizona_plants_unified_20251018.jsonl``` file found in the ```data-preparation``` directory.

## Steps:
1. Load the documents

In [ ]:
from openai import OpenAI

vector_store = ArizonaPlantVectorStore()
assistant = ArizonaPlantRAG(vector_store, openai_client)


# Create evaluator
evaluator = RAGEvaluator(
     openai_client=openai_client,
     vector_store=vector_store,
     rag_function=assistant.rag
)

# load documents
documents = load_dataset('data-preparation/arizona_plants_unified_20251018.jsonl')

2. Generate our ground truth dataset -- **THIS ONLY NEEDS TO RUN ONCE UNLESS FIXING SOMETHING ELSE**
    1. We are generating 3 questions for 50 documents to obtain statistically meaningful metrics

In [26]:
ground_truth = evaluator.generate_ground_truth_dataset(
    documents=documents,
    questions_per_doc=3,
    max_documents=50  # Start small
)

with open('evaluation/ground_truth_qa.json', 'w') as f:
    json.dump(ground_truth, f, indent=2)


Loading embedding model: all-MiniLM-L6-v2
✓ Model loaded (dimension: 384)
✓ Qdrant client initialized

3. Loading dataset from: data-preparation/arizona_plants_unified_20251018.jsonl
   ✓ Loaded 340 documents
Generating ground truth dataset from 50 documents...


Generating questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:35<00:00,  4.32s/it]

✓ Generated 150 question-answer pairs


3. Run the evaluation using LLM as a judge
    1. This step takes ~ **30 minutes**

In [47]:
ground_truth[10]

{'question': 'Is the tree yucca native to Arizona or another region?',
 'ground_truth_answer': 'The tree yucca, or Yucca filifera, is native to central Mexico. Although there are 3,545 observations recorded in Arizona, it is not native to that region.',
 'source_doc_id': 'species_290831',
 'source_doc_title': 'tree yucca (Yucca filifera)',
 'document_type': 'species'}

In [48]:
# Load the existing ground truth
with open('evaluation/ground_truth_qa.json', 'r') as f:
    ground_truth = json.load(f)

# Run evaluation
results_df = evaluator.evaluate_rag_system(ground_truth)

# Analyze results
print("\nLowest scoring questions:")
print(results_df.nsmallest(5, 'average_score')[['question', 'average_score']])


Evaluating RAG system on 150 questions...


Evaluating:   0%|                                                                                                                                                                                                                                                                                                                                 | 0/150 [00:00<?, ?it/s]

Searching for: "What are the common names for Opuntia macrorhiza?"
✓ Found 5 results

RAG Pipeline
Searching for: "What are the common names for Opuntia macrorhiza?"
✓ Found 5 results
Generating answer...
✓ Answer generated

Searching for: "What are the common names for Opuntia macrorhiza?"
✓ Found 5 results


Evaluating:   1%|██                                                                                                                                                                                                                                                                                                                       | 1/150 [00:02<06:41,  2.69s/it]

Searching for: "In which regions of the United States can prairie pricklypear be found?"
✓ Found 5 results

RAG Pipeline
Searching for: "In which regions of the United States can prairie pricklypear be found?"
✓ Found 5 results
Generating answer...
✓ Answer generated

Searching for: "In which regions of the United States can prairie pricklypear be found?"
✓ Found 5 results


...

Searching for: "How might the presence of the small baileya moth affect my garden plants?"
✓ Found 5 results

RAG Pipeline
Searching for: "How might the presence of the small baileya moth affect my garden plants?"
✓ Found 5 results
Generating answer...
✓ Answer generated

Searching for: "How might the presence of the small baileya moth affect my garden plants?"
✓ Found 5 results


Evaluating:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 149/150 [24:45<00:08,  8.74s/it]

Searching for: "Is Baileya australis attracted to specific types of plants commonly found in Arizona gardens?"
✓ Found 5 results

RAG Pipeline
Searching for: "Is Baileya australis attracted to specific types of plants commonly found in Arizona gardens?"
✓ Found 5 results
Generating answer...
✓ Answer generated

Searching for: "Is Baileya australis attracted to specific types of plants commonly found in Arizona gardens?"
✓ Found 5 results


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [24:48<00:00,  9.92s/it]


EVALUATION RESULTS

📊 Retrieval Metrics:
  Hit Rate@5: 96.00%
  Mean Reciprocal Rank: 0.919

📝 Answer Quality (Average scores out of 5):
  Relevance:     4.88
  Faithfulness:  4.93
  Completeness:  4.60
  Clarity:       4.97
  Overall:       4.84

✓ Detailed results saved to: rag_evaluation_results.csv

Lowest scoring questions:
                                              question  average_score
81   What are the common names for Cylindropuntia f...           2.75
5    What is another common name for Cylindropuntia...           3.25
20   Is Penstemon digitalis suitable for Arizona ga...           3.75
147  What are the typical habitats where Baileya au...           3.75
149  Is Baileya australis attracted to specific typ...           3.75
